In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import config
import utils
import reader

In [32]:
## util
import os
import logging
from argparse import ArgumentParser
from tqdm import tqdm_notebook as tqdmnb
from tqdm import tqdm as tqdm
import pickle
import json 
import jsonlines as jsonl
from collections import defaultdict
from typing import Iterable, List, Dict, Tuple, Union
from pathlib import Path
## graph
import networkx as nx
import matplotlib.pyplot as plt
## nn
import numpy as np
import torch
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data.data import Data
## Stanza
import stanza
from stanza.models.common.doc import Document
from stanza.pipeline.core import Pipeline
## allennlp model
from allennlp_models.structured_prediction.predictors.srl import SemanticRoleLabelerPredictor
from allennlp_models.structured_prediction.predictors.biaffine_dependency_parser import BiaffineDependencyParserPredictor
from allennlp.predictors.predictor import Predictor #
## allennlp
from allennlp.data import Token, Vocabulary, Instance
from allennlp.data.fields import ListField, TextField, Field
from allennlp.data.token_indexers import (
    SingleIdTokenIndexer,
    TokenCharactersIndexer,
    ELMoTokenCharactersIndexer,
    PretrainedTransformerIndexer,
    PretrainedTransformerMismatchedIndexer,
)
from allennlp.data import DatasetReader, DataLoader, Instance, Vocabulary
from allennlp.data.tokenizers import (
    CharacterTokenizer,
    PretrainedTransformerTokenizer,
    SpacyTokenizer,
    WhitespaceTokenizer,
)
from allennlp.modules.seq2vec_encoders import CnnEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import (
    Embedding,
    TokenCharactersEncoder,
    ElmoTokenEmbedder,
    PretrainedTransformerEmbedder,
    PretrainedTransformerMismatchedEmbedder,
)
from allennlp.nn import util as nn_util

In [4]:
file_path = config.P_TEST_FILE
print(file_path)

/work/2020-IIS-NLU-internship/MNLI/data/MNLI_Stanza/pre_multinli_1.0_dev_mismatched.jsonl


In [15]:
from sparse_adjacency_field import SparseAdjacencyField as SF

In [17]:
pygd = Data(edge_index=torch.tensor([[0,1], [1,0]]), edge_attr=["e0", "e1"])
pygd
seqf = TextField([Token("v0"), Token("vi")], token_indexers={"tokens":SingleIdTokenIndexer(namespace='tokens')})
seqf2 = TextField([Token("v2"), Token("vi"), Token("v0")], token_indexers={"tokens":SingleIdTokenIndexer(namespace='tokens')})

In [18]:
pygd2 = Data(edge_index=torch.tensor([[0,1, 2], [1,0,2]]), edge_attr=["e2", "e1", "e3"])
pygd2

Data(edge_attr=[3], edge_index=[2, 3])

In [19]:
spaf = SF(pygd, sequence_field=seqf)

In [20]:
spaf2 = SF(pygd2, sequence_field=seqf2)

In [21]:
vocab2 = Vocabulary.from_instances(instances=[spaf, spaf2])

In [22]:
#bat = Batch([spaf, spaf2])
print(vocab2.get_namespaces())

{'edge_labels'}


In [23]:
spaf.index(vocab2)
spaf2.index(vocab2)

In [24]:
t1 = spaf.as_tensor(spaf.get_padding_lengths())
t2 = spaf2.as_tensor(spaf.get_padding_lengths())
print(t1, t2, sep='\n')

{'edge_attr': tensor([1, 0]), 'edge_index': tensor([[0, 1],
        [1, 0]]), 'batch_id': tensor([0., 0.])}
{'edge_attr': tensor([2, 0, 3]), 'edge_index': tensor([[0, 1, 2],
        [1, 0, 2]]), 'batch_id': tensor([0., 0., 0.])}


In [25]:
bt3 = spaf.batch_tensors([t1, t2, t1])
bt3

{'edge_index': tensor([[0, 1, 2, 3, 4, 5, 6],
         [1, 0, 3, 2, 4, 6, 5]]),
 'edge_attr': tensor([1, 0, 2, 0, 3, 1, 0]),
 'batch_id': tensor([0, 0, 1, 1, 1, 2, 2])}

In [26]:
spaf.empty_field()

In [41]:
del reader 
import reader

In [42]:
rdr = reader.NLI_Graph_Reader()

In [43]:
dev_set = rdr.read(config.P_DEV_FILE)

In [44]:
vocab = Vocabulary.from_instances(dev_set)

In [79]:
vocab.print_statistics()



----Vocabulary Statistics----


Top 10 most frequent tokens in namespace 'edge_labels':
	Token: punct		Frequency: 32065
	Token: case		Frequency: 30338
	Token: nsubj		Frequency: 29153
	Token: det		Frequency: 28436
	Token: root		Frequency: 20496
	Token: advmod		Frequency: 17098
	Token: amod		Frequency: 16704
	Token: obj		Frequency: 15775
	Token: obl		Frequency: 14845
	Token: compound		Frequency: 13444

Top 10 longest tokens in namespace 'edge_labels':
	Token: compound:prt		length: 12	Frequency: 1258
	Token: nsubj:pass		length: 10	Frequency: 2960
	Token: det:predet		length: 10	Frequency: 256
	Token: nmod:npmod		length: 10	Frequency: 140
	Token: cc:preconj		length: 10	Frequency: 122
	Token: reparandum		length: 10	Frequency: 10
	Token: csubj:pass		length: 10	Frequency: 7
	Token: nmod:poss		length: 9	Frequency: 5120
	Token: acl:relcl		length: 9	Frequency: 3045
	Token: discourse		length: 9	Frequency: 2649

Top 10 shortest tokens in namespace 'edge_labels':
	Token: cc		length: 2	Frequency: 9

In [58]:
dev_set.index_with(vocab)

In [74]:
dev_loader = DataLoader(dev_set, batch_size=32)

In [75]:
batch = next(iter(dev_loader))

In [82]:
batch["g_p"].keys()

dict_keys(['edge_index', 'edge_attr', 'batch_id'])

In [72]:
import model

NameError: name 'GraphConvLayer' is not defined